In [ ]:
#Refrence Link: https://github.com/priya-dwivedi/Deep-Learning/blob/master/GrammarCorrector/T5_Grammar.ipynb

In [1]:
!nvidia-smi

Thu Mar  2 21:48:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.84       Driver Version: 460.84       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   29C    P8     8W / 250W |    447MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 20%   

In [2]:
import os
import torch
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, "0" to  "7" 
os.environ["CUDA_VISIBLE_DEVICES"]="3"
#torch.cuda.set_device(0)
n_gpu = torch.cuda.device_count()
print(n_gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)


1


'GeForce GTX 1080 Ti'

In [3]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm


In [4]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
#Set a seed
import random
import numpy as np
import torch
import datasets
import pandas as pd

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /DATA/gupta92/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(42)

In [6]:
#Loading dataset

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
df = pd.read_csv('DATA/etoori_train.csv')
df.shape


(140000, 2)

In [9]:
df.rename(columns = {'enc_input':'input'}, inplace = True)
df.rename(columns = {'dec_input':'output'}, inplace = True)

In [10]:
df.head(10)

,input,output
0,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अप हृदय में गाँठ बनाकर नहीं रखते थे ।,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अपने हृदय में गाँठ बनाकर नहीं रखते थे ।
1,देश में हिन्दी को विस्थापित कर का षड़यंत्र चल रहा है जो चिंता का विषय है ।,देश में हिन्दी को विस्थापित करने का षड़यंत्र चल रहा है जो चिंता का विषय है ।
2,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर लग जा के बाद डेरा प्रेमी इस बार दर्ज हुए मुकदमे में सभी अभियुक्तों की गिरफ्तारी की मांग को लेकर आंदोलनरत हैं ।,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर लग जाने के बाद डेरा प्रेमी इस बार दर्ज हुए मुकदमे में सभी अभियुक्तों की गिरफ्तारी की मांग को लेकर आंदोलनरत हैं ।
3,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम से सिंगापुर के पेरानांका म्यूजियम में प्रदर्शनी लगाई गई हैं ।,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम से सिंगापुर के पेरानांका म्यूजियम में प्रदर्शनी लगाई गई है ।
4,तब तक के लिए हमें विराम ले की अनुमति दीजिए ।,तब तक के लिए हमें विराम लेने की अनुमति दीजिए ।
5,देव स्थेन तक सड़क बनाने की परियोजना चल रही है ।,देव स्थान तक सड़क बनाने की परियोजना चल रही है ।
6,वर्ड वाचर व प्रसिद्ध चित्रकार अनूप साह आरएनएस को बताया कि श्वेतकंठ चिलचिल पक्षी का वैज्ञानिक नाम ह्वाइट थ्रोटेट लाफिंग थ्रस है ।,वर्ड वाचर व प्रसिद्ध चित्रकार अनूप साह ने आरएनएस को बताया कि श्वेतकंठ चिलचिल पक्षी का वैज्ञानिक नाम ह्वाइट थ्रोटेट लाफिंग थ्रस है ।
7,दो दिन पहले ज्ञानजी पोस्ट ठेली जिसमें,दो दिन पहले ज्ञानजी ने पोस्ट ठेली जिसमें
8,दुर्घटना की स्थिति में बीमा कम्पनी द्वारा दी जा वाली राशि निर्धारित कर दी गई है ।,दुर्घटना की स्थिति में बीमा कम्पनी द्वारा दी जाने वाली राशि निर्धारित कर दी गई है ।
9,देर से जाऊंगा तो दुकां बंद हो जाएंगी ।,देर से जाऊंगा तो दुकानें बंद हो जाएंगी ।


In [11]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")



/usr/lib/python3/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [13]:
tokenizer.model_max_length

512

In [17]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from transformers import EncoderDecoderModel, BertTokenizer

#model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")
from transformers import EncoderDecoderModel, BertTokenizer
import torch

model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    "bert-base-multilingual-cased", "bert-base-multilingual-cased"
)  # initialize Bert2Bert from pre-trained checkpoints

# training
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.wei

In [18]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [19]:
#Train - Test Split
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((126000, 2), (14000, 2))

In [20]:
test_df['input_token_len'] = test_df['input'].apply(calc_token_len)

In [21]:
test_df.head()

,input,output,input_token_len
40665,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है ।,36
48520,तथे गौशाला का,तथा गौशाला का,9
138403,बच्चे खेल रहे है किलकारियाँ गूंज रही है ब्रह्माण्ड में,बच्चे खेल रहे हैं किलकारियाँ गूंज रही हैं ब्रह्माण्ड में,25
130079,धरने पर किसानो ने कहा कि अब समय आ गया हैं कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी हैं ।,धरने पर किसानो ने कहा कि अब समय आ गया है कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी है ।,44
50146,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा हैं ।,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा है ।,37


In [22]:
test_df['input_token_len'].describe()

count    14000.000000
mean        34.456571
std         20.822809
min          4.000000
25%         21.000000
50%         30.000000
75%         43.000000
max        474.000000
Name: input_token_len, dtype: float64

In [23]:
#We will use a token length of 64 since it will cover the vast majority of examples
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [24]:
test_dataset

Dataset({
    features: ['input', 'output', 'input_token_len', '__index_level_0__'],
    num_rows: 14000
})

In [25]:
#Load the Dataset
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):         
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64
  
    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['output']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)
    
        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }
        
        return inputs

  
    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])
        
        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [26]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[10])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 50
attention_mask 50
labels 50
{'input_ids': [101, 11081, 11549, 19741, 27391, 12213, 67147, 10914, 868, 14070, 13432, 18187, 18438, 35247, 19966, 13088, 865, 18351, 73649, 11208, 16192, 37444, 17798, 10977, 89652, 893, 78530, 17110, 77386, 11267, 889, 24667, 70288, 19255, 14265, 868, 14070, 13432, 18187, 18438, 35247, 10826, 866, 41082, 14080, 16791, 893, 34315, 920, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [101, 11081, 11549, 19741, 27391, 12213, 67147, 10914, 868, 14070, 13432, 18187, 18438, 35247, 19966, 13088, 865, 18351, 73649, 11208, 16192, 37444, 13794, 10977, 89652, 893, 78530, 17110, 77386, 11267, 889, 24667, 70288, 19255, 14265, 868, 14070, 13432, 18187, 18438, 35247, 10826, 866, 41082, 14080, 16791, 893, 34315, 920, 102]}


In [27]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[0])

input_ids 36
attention_mask 36
labels 36
{'input_ids': [101, 851, 28462, 45086, 73184, 35773, 10914, 898, 17110, 88547, 10412, 15236, 10532, 895, 14251, 15070, 108775, 29161, 13220, 19810, 63214, 15837, 81583, 59444, 13286, 64897, 895, 92943, 13432, 27155, 66326, 27373, 36072, 10569, 920, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [101, 851, 28462, 45086, 73184, 35773, 10914, 898, 17110, 88547, 10412, 15236, 10532, 895, 14251, 15070, 108775, 29161, 13220, 19810, 63214, 15837, 81583, 59444, 13286, 79610, 895, 92943, 13432, 27155, 66326, 27373, 36072, 10569, 920, 102]}


In [28]:
#Define Evaluator
#!pip install rouge_score

In [29]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

In [30]:
#Train Model
#data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')


In [31]:
#model_name = 't5_gec_hindi_muRIL_1'
#torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 
#model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [32]:
# defining training related arguments
batch_size = 8
args = Seq2SeqTrainingArguments(
            output_dir="models/hindi/EncDec_bert",
            evaluation_strategy="epoch",
            learning_rate=2e-5,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            weight_decay=0.1,
            save_steps=10000,
            num_train_epochs=2,
            predict_with_generate=True,
            fp16=True
        )

In [33]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
#os.environ["CUDA_LAUNCH_BLOCKING"] = "false"
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [34]:
import torch
#torch.cuda.empty_cache()

In [35]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator)
                #compute_metrics=compute_metrics)

Using cuda_amp half precision backend


In [36]:
#import torch
#torch.cuda.empty_cache()

In [37]:
#Wandb API Key: bcdbdd5ee9d76a20c90b5f2a246eb45f14b341a9
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
trainer.train()

***** Running training *****
  Num examples = 126000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 31500
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch,Training Loss,Validation Loss
1,0.052400,0.089137
2,0.022100,0.224824


Saving model checkpoint to models/hindi/EncDec_bert/checkpoint-10000
Configuration saved in models/hindi/EncDec_bert/checkpoint-10000/config.json
Model weights saved in models/hindi/EncDec_bert/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in models/hindi/EncDec_bert/checkpoint-10000/tokenizer_config.json
Special tokens file saved in models/hindi/EncDec_bert/checkpoint-10000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 14000
  Batch size = 8
Saving model checkpoint to models/hindi/EncDec_bert/checkpoint-20000
Configuration saved in models/hindi/EncDec_bert/checkpoint-20000/config.json
Model weights saved in models/hindi/EncDec_bert/checkpoint-20000/pytorch_model.bin
tokenizer config file saved in models/hindi/EncDec_bert/checkpoint-20000/tokenizer_config.json
Special tokens file saved in models/hindi/EncDec_bert/checkpoint-20000/special_tokens_map.json
Saving model checkpoint to models/hindi/EncDec_bert/checkpoint-30000
Configuration saved in

TrainOutput(global_step=31500, training_loss=0.32566606230962847, metrics={'train_runtime': 11218.625, 'train_samples_per_second': 22.463, 'train_steps_per_second': 2.808, 'total_flos': 1.714918853611512e+16, 'train_loss': 0.32566606230962847, 'epoch': 2.0})

In [ ]:
#torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [38]:
#Save the trained Model
trainer.save_model('hindi_EncDec_mbert')

Saving model checkpoint to hindi_EncDec_mbert
Configuration saved in hindi_EncDec_mbert/config.json
Model weights saved in hindi_EncDec_mbert/pytorch_model.bin
tokenizer config file saved in hindi_EncDec_mbert/tokenizer_config.json
Special tokens file saved in hindi_EncDec_mbert/special_tokens_map.json


In [ ]:
model

# Testing Trained Model

In [39]:
#Testing
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,AutoTokenizer
model_name = 'hindi_EncDec_mbert'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 

tokenizer1 = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")


loading configuration file config.json from cache at /DATA/gupta92/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "voca

In [70]:

from transformers import EncoderDecoderModel
model_name = 'hindi_EncDec_mbert'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 
# load fine-tuned model
trained_model = EncoderDecoderModel.from_pretrained(model_name).to(torch_device)

loading configuration file hindi_EncDec_mbert/config.json
Model config EncoderDecoderConfig {
  "_commit_hash": null,
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "bert-base-multilingual-cased",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0

In [84]:
trained_model

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [87]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(trained_model)

+----------------------------------------------------------------------+------------+
|                               Modules                                | Parameters |
+----------------------------------------------------------------------+------------+
|              encoder.embeddings.word_embeddings.weight               |  91812096  |
|            encoder.embeddings.position_embeddings.weight             |   393216   |
|           encoder.embeddings.token_type_embeddings.weight            |    1536    |
|                 encoder.embeddings.LayerNorm.weight                  |    768     |
|                  encoder.embeddings.LayerNorm.bias                   |    768     |
|         encoder.encoder.layer.0.attention.self.query.weight          |   589824   |
|          encoder.encoder.layer.0.attention.self.query.bias           |    768     |
|          encoder.encoder.layer.0.attention.self.key.weight           |   589824   |
|           encoder.encoder.layer.0.attention.self.key

384194811

In [86]:
#!pip3 install prettytable

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [88]:
param_size = 0
for param in trained_model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in trained_model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 1465.603MB


In [91]:
!pip3 install torchview

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [71]:
def correct_grammar(input_text,num_return_sequences):
    #batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
    batch = tokenizer1([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt",return_token_type_ids=False).to(device)
    #print(batch)
    #translated = model1.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    translated= trained_model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    #print("here")
    tgt_text = tokenizer1.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [72]:
#print(T5ForConditionalGeneration.__version__)

In [73]:
text = test_df['input'].iat[0]
correct = test_df['output'].iat[0]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=4)[0]
p=len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।
Correct Seentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है । 
Predicted Sentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है । 


In [74]:
text = test_df['input'].iat[13]
correct = test_df['output'].iat[13]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=2)[0]
p = len(predicted_s)

if(p>l):
    predicted_s = predicted_s[:l]


print("Predicted Sentence:",predicted_s)

Incorrect Sentence: प्रत्येक शिक्षक जानता हैं कि उसके मासिक वेतन का कुछ अंश घूस में जा रहा हैं ।
Correct Seentence: प्रत्येक शिक्षक जानता है कि उसके मासिक वेतन का कुछ अंश घूस में जा रहा है । 
Predicted Sentence: प्रत्येक शिक्षक जानता है कि उसके मासिक वेतन का कुछ अंश घूस में जा रहा है । 


In [75]:
text = test_df['input'].iat[43]
correct = test_df['output'].iat[43]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=2)[0]
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: इस सबमें अहम बात मुस्लिम मतदाताओं का रुझान हैं ।
Correct Seentence: इस सबमें अहम बात मुस्लिम मतदाताओं का रुझान है । 
Predicted Sentence: इस सबमें अहम बात मुस्लिम मतदाताओं का रुझान है । 


In [76]:
text = test_df['input'].iat[1]
correct = test_df['output'].iat[1]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=4)[0][:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: तथे गौशाला का
Correct Seentence: तथा गौशाला का
Predicted Sentence: तथा गौशाला का


In [77]:
text = test_df['input'].iat[0]
correct = test_df['output'].iat[0]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=2)[0]
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।
Correct Seentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है । 
Predicted Sentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है । 


In [79]:
text = test_df['input'].iat[51]
correct = test_df['output'].iat[51]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: गनीमत हैं गणेशशंकर विद्यार्थी ने २०० रुपये का मनीआर्डर भेजकर अशफ़ाक़ की मजार पर छत डलवा कर उसे धूप के साये से बचा लिया ।
Correct Seentence: गनीमत है गणेशशंकर विद्यार्थी ने २०० रुपये का मनीआर्डर भेजकर अशफ़ाक़ की मजार पर छत डलवा कर उसे धूप के साये से बचा लिया । 
Predicted Sentence: गनीमत है गणेशशंकर विद्यार्थी ने २०० रुपये का मनीआर्डर भेजकर अशफ़ाक़ की मजार पर छत डलवा कर उसे धूप के साये से बचा लिया । 


In [80]:
text = test_df['input'].iat[60]
correct = test_df['output'].iat[60]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)


Incorrect Sentence: ज़ाहिर है जब मैंने लिखूंगा तो वो मेरे ही व्यक्तिगत विचार कहलायेंगे किसी संस्थान के नहीं ।
Correct Seentence: ज़ाहिर है जब मैं लिखूंगा तो वो मेरे ही व्यक्तिगत विचार कहलायेंगे किसी संस्थान के नहीं । 
Predicted Sentence: ज़ाहिर है जब मैं लिखूंगा तो वो मेरे ही व्यक्तिगत विचार कहलायेंगे किसी संस्थान के नहीं । 


In [97]:
text = test_df['input'].iat[51]
correct = test_df['output'].iat[51]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: गनीमत हैं गणेशशंकर विद्यार्थी ने २०० रुपये का मनीआर्डर भेजकर अशफ़ाक़ की मजार पर छत डलवा कर उसे धूप के साये से बचा लिया ।
Correct Seentence: गनीमत है गणेशशंकर विद्यार्थी ने २०० रुपये का मनीआर्डर भेजकर अशफ़ाक़ की मजार पर छत डलवा कर उसे धूप के साये से बचा लिया । 
Predicted Sentence: गनीमत है गणेशशंकर विद्यार्थी ने २०० रुपये का मनीआर्डर भेजकर अशफ़ाक़ की मजार पर छत डलवा कर उसे धूप के साये से बचा लिया । 


In [98]:
%%time
predicted_s = correct_grammar(text, num_return_sequences=1)[0]

CPU times: user 1.12 s, sys: 23.9 ms, total: 1.14 s
Wall time: 1.14 s


In [96]:
text = test_df['input'].iat[20]
correct = test_df['output'].iat[20]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
p=len(predicted_s)
l = len(text)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: हम पास खींच लेते हैंं ।
Correct Seentence: हम पास खींच लेते हैं । 
Predicted Sentence: हम पास खींच लेते हैं । 


# Performance Analysis
1. BLEU Score
2. GLEU Score

In [52]:
test_df.shape

(14000, 3)

In [53]:
test_df.head(5)

,input,output,input_token_len
40665,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है ।,36
48520,तथे गौशाला का,तथा गौशाला का,9
138403,बच्चे खेल रहे है किलकारियाँ गूंज रही है ब्रह्माण्ड में,बच्चे खेल रहे हैं किलकारियाँ गूंज रही हैं ब्रह्माण्ड में,25
130079,धरने पर किसानो ने कहा कि अब समय आ गया हैं कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी हैं ।,धरने पर किसानो ने कहा कि अब समय आ गया है कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी है ।,44
50146,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा हैं ।,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा है ।,37


In [54]:
import pandas as pd
df_test = pd.read_csv('DATA/etoori_test.csv')
df_test.shape

(30000, 2)

In [55]:
df_test.head(5)

,enc_input,dec_input
0,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार भी को मिलियन डॉलर क्लब में माना जा रहा है ।,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है ।
1,यह मन को काबू में करने वाली मुद्रा हैं इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैंं ।,यह मन को काबू में करने वाली मुद्रा है इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैं ।
2,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा उसनेाँ पर शोर मचा रहा है ।,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ पर शोर मचा रहा है ।
3,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला हैं ।,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला है ।
4,उनकी वो वाली बात भी अनिश्चित रहती हैं ।,उनकी वो वाली बात भी अनिश्चित रहती है ।


In [56]:
text = df_test['enc_input'].iat[0]
correct = df_test['dec_input'].iat[0]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार भी को मिलियन डॉलर क्लब में माना जा रहा है ।
Correct Seentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है । 
Predicted Sentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है । 


In [57]:
text = df_test['enc_input'].iat[1]
correct = df_test['dec_input'].iat[1]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: यह मन को काबू में करने वाली मुद्रा हैं इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैंं ।
Correct Seentence: यह मन को काबू में करने वाली मुद्रा है इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैं । 
Predicted Sentence: यह मन को काबू में करने वाली मुद्रा है इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैं । 


In [58]:
text = df_test['enc_input'].iat[2]
correct = df_test['dec_input'].iat[2]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा उसनेाँ पर शोर मचा रहा है ।
Correct Seentence: आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ पर शोर मचा रहा है । 
Predicted Sentence: आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ पर शोर मचा रहा है । 


In [59]:
text = df_test['enc_input'].iat[3]
correct = df_test['dec_input'].iat[3]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला हैं ।
Correct Seentence: परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला है । 
Predicted Sentence: परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला है । 


In [60]:
text = df_test['enc_input'].iat[4]
correct = df_test['dec_input'].iat[4]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: उनकी वो वाली बात भी अनिश्चित रहती हैं ।
Correct Seentence: उनकी वो वाली बात भी अनिश्चित रहती है । 
Predicted Sentence: उनकी वो वाली बात भी अनिश्चित रहती है । 


In [61]:
text = df_test['enc_input'].iat[9]
correct = df_test['dec_input'].iat[9]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: मैंने ललित हूँ ।
Correct Seentence: मैं ललित हूँ । 
Predicted Sentence: मैं ललित हूँ । 


In [64]:
from nltk.translate.gleu_score import sentence_gleu
GLEU_val_emb = []
test_data = df_test.head(10000)
print(test_data.shape)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%1000 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(10000, 2)


1it [00:01,  1.10s/it]

GELU Score for  1  data point 1.0


1001it [19:12,  1.19s/it]

GELU Score for  1001  data point 0.9146081119649399


...

GELU Score for  8001  data point 0.9165724757187045


9001it [2:53:46,  1.15s/it]

GELU Score for  9001  data point 0.916987191679546


10000it [3:12:49,  1.16s/it]

GELU Score =  0.9175768315121448


In [95]:
print("GELU Score = ",np.mean(GLEU_val_emb))

GELU Score =  0.9175768315121448
